# Implementing Tok-k Importance Routing 

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Expert(nn.Module):
    def __init__(self, d_model, d_hidden, activation, dropout):
        super().__init__()
        act = {"relu":F.relu(), "gelu": F.gelu()}[activation]
        self.ffn = nn.ModuleList(
            nn.Sequential(
                nn.Linear(d_model, d_hidden),
                act,
                nn.Dropout(dropout),
                nn.Linear(d_hidden, d_model)
            )
        )

    def forward(self, x):
        return self.ffn(x)

### MoE Layer With Importance Routing

In [ ]:
class MoE(nn.Module):
    def __init__(self):
        super().__init__()

    
    def forward(self, x):
        """ 
        x : (B, S, D)
        """
        B, S, D = x.shape
        T = B * S
        x_flat = x.view(T, D)

        

SyntaxError: incomplete input (3155914306.py, line 1)